In [1]:
!pip install asttokens==2.1.0 bleach==3.3.0 certifi==2020.12.5 cffi==1.14.4 chardet==4.0.0 colorama==0.4.4 cryptography==3.3.1 cycler==0.10.0 dnspython==2.2.1 docutils==0.16 executing==1.2.0 icecream==2.1.3 idna==2.10 importlib-resources==5.10.0 jeepney==0.6.0 keyring==22.0.1 kiwisolver==1.3.1 matplotlib==3.3.4 numpy==1.19.5 opencv-python==4.5.1.48 packaging==20.9 Pillow==8.1.0 pkginfo==1.7.0 pycocotools==2.0.6 pycparser==2.20 Pygments==2.7.4 pymongo==4.3.2 pyparsing==2.4.7 python-dateutil==2.8.1 readme-renderer==28.0 requests==2.25.1 requests-toolbelt==0.9.1 rfc3986==1.4.0 SecretStorage==3.3.0 six==1.15.0 torch==1.7.1 torchray==1.0.0.2 torchvision==0.8.2 tqdm==4.56.0 twine==3.3.0 typing-extensions==3.7.4.3 urllib3==1.26.3 webencodings==0.5.1 zipp==3.10.0

  Obtaining dependency information for asttokens==2.1.0 from https://files.pythonhosted.org/packages/23/f7/19e20888d0b1e44b40c2795894a05e7be1f631c09949f7e0b177df1ab7a2/asttokens-2.1.0-py2.py3-none-any.whl.metadata
  Using cached asttokens-2.1.0-py2.py3-none-any.whl.metadata (4.8 kB)
  Obtaining dependency information for certifi==2020.12.5 from https://files.pythonhosted.org/packages/5e/a0/5f06e1e1d463903cf0c0eebeb751791119ed7a4b3737fdc9a77f1cdfb51f/certifi-2020.12.5-py2.py3-none-any.whl.metadata
  Using cached certifi-2020.12.5-py2.py3-none-any.whl.metadata (3.0 kB)
  Using cached cffi-1.14.4.tar.gz (471 kB)
  Preparing metadata (setup.py) ... done
  Obtaining dependency information for colorama==0.4.4 from https://files.pythonhosted.org/packages/44/98/5b86278fbbf250d239ae0ecb724f8572af1c91f4a11edf4d36a206189440/colorama-0.4.4-py2.py3-none-any.whl.metadata
  Using cached colorama-0.4.4-py2.py3-none-any.whl.metadata (14 kB)
  Using cached cryptography-3.3.1.tar.gz (539 kB)
  Installing

In [ ]:
!pip install asttokens==2.1.0 bleach==3.3.0 certifi==2020.12.5

In [ ]:
!pip install --upgrade certifi

In [ ]:
!pip install kaggle==1.6.11

In [ ]:
!git clone https://github.com/nguyenphuc22/TorchPRISM.git

In [ ]:
import sys
sys.path.append('/content/TorchPRISM')

In [ ]:
!pip install icecream

In [ ]:
import torch
import numpy as np
import torch.nn as nn
import os, glob
from torchvision import transforms
from torchvision import models
from torchprism import PRISM
import matplotlib.pyplot as plt
from PIL import Image
import json


with open("/content/TorchPRISM/classes.json") as json_file:
    CLASSES = json.load(json_file)

CLASSES_IDs = { int(k): v.split(",")[0].replace(" ", "_") for k,v in CLASSES.items() }
CLASSES_NAMEs = { v.split(",")[0].replace(" ", "_").lower(): int(k) for k,v in CLASSES.items() }

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

def read_images_2_batch():
    image_files = glob.glob("/content/TorchPRISM/samples/*.jpg")
    image_files.sort()

    input_images = [Image.open(f) for f in image_files]
    input_batch = torch.stack([transform(image) for image in input_images])

    return image_files, input_images, input_batch

def prepare_network(arch):
    model = models.__dict__[arch](pretrained=True)
    model.eval()
    PRISM.register_hooks(model)
    return model

def print_output(output, image_files_names):
    listed_output = [ { CLASSES_IDs[i]: val for i, val in enumerate(o.tolist()) } for o in output ]
    classification = []
    for i, name in enumerate(image_files_names):
        print(f"\n{name}:")
        classes = []
        for k, v in sorted(listed_output[i].items(), key=lambda o: o[1], reverse=True)[:5]:
            # print(f"{k}: {v:.2f}")
            classes.append(f"{k}: {v:.2f}")
        classification.append(classes)
    return classification

def normalize_image(image):
    # change image tensor from -1,1 to 0,1
    return (image - image.min()) / (image.max() - image.min())

if __name__ == "__main__":
    arches = [
        "vgg16",
        # "vgg11",
        # "vgg16",
        # "vgg19",
        # "resnet18",
        # "resnet50",
        # "resnet101",
        # "googlenet",
        # "alexnet",
        # "mobilenet_v2",
        # "squeezenet1_0"
    ]

    for arch in arches:
        with torch.no_grad():
            print(arch)
            PRISM.prune_old_hooks(None)
            model = prepare_network(arch)

            image_files_names, input_images, input_batch = read_images_2_batch()

            if torch.cuda.is_available():
                print("Running on GPU")
                input_batch = input_batch.to("cuda")
                model.to("cuda")


            output = model(input_batch)
            percentage = nn.Softmax(dim=1)
            # print(f"SHAPE {output.shape}")
            classification = print_output(percentage(output), image_files_names)

            prism_maps = PRISM.get_maps().permute(0, 2, 3, 1).detach().cpu().numpy()

            columns = input_batch.shape[0]
            fig, ax = plt.subplots(nrows=2, ncols=columns)
            input_batch = input_batch.permute(0, 2, 3, 1).detach().cpu().numpy()

            if columns == 1:
                ax[0].imshow(normalize_image(input_batch[0]))
                ax[0].set_title("\n".join(classification[0]), fontsize=3)
                ax[0].axis('off')
                ax[1].imshow(prism_maps[0])
                ax[1].axis('off')
            else:
                for column in range(columns):
                    ax[0][column].imshow(normalize_image(input_batch[column]))
                    ax[0][column].set_title("\n".join(classification[column]), fontsize=3)
                    ax[0][column].axis('off')

                for column in range(columns):
                    ax[1][column].imshow(prism_maps[column])
                    ax[1][column].axis('off')

            fig.suptitle(f'PRISM', fontsize=10)
            fig.tight_layout()
            fig.subplots_adjust(top=0.99)
            plt.savefig(f"results/PRISM_{arch}.jpg", format='jpg', bbox_inches="tight", dpi=500)
